## Fast computation algorithm for the Rayleigh–Sommerfeld diffraction formula Example

#### 2020-12-16

In [1]:
import numpy as np
import scipy.integrate as integrate
import os,sys
import yaml
import matplotlib.pyplot as plt
import math
from ipywidgets import *

In [2]:
with open('configure.yml','r') as conf_para:
    conf_para = yaml.load(conf_para,Loader=yaml.FullLoader)
print(conf_para)

{'Source': {'p0': 200000.0, 'wl': 7.29e-11, 'th_s': 0.0002}, 'Exp_geom': {'defocus': 0.004, 'det_dist': 0.014}, 'Detector': {'fs_size': 2000, 'ss_size': 2000, 'pixelsize_x': 5.5e-05, 'pixelsize_y': 5.5e-05}, 'Lens': {'focus_x': 0.0012, 'focus_y': 0.001, 'ap_x': 4e-05, 'ap_y': 4e-05, 'alpha_x': -0.05, 'alpha_y': -0.05, 'x_abcen': 0.5, 'y_abcen': 0.5}}


In [3]:
n_x = 100000
ap_x = 40e-6
wl = conf_para['Source']['wl']
focus = conf_para['Lens']['focus_x']
defocus = conf_para['Exp_geom']['defocus']
alpha = -0.05
n_arr = np.arange(-n_x // 2, n_x // 2)
x0_arr = n_arr * ap_x / n_x

u0 = np.exp(1j * np.pi * x0_arr**2 / wl / focus + 1e9j * alpha * (x0_arr / focus)**3)
%matplotlib widget
#plt.plot(u0)
#u0[np.abs(x0_arr) > ap_x/2]=0
#plt.plot(u0)

In [11]:
u0_f = np.fft.fft(u0)/n_x
delta_x = ap_x/n_x
h_f = np.fft.fft(delta_x * (focus + defocus) / 1j / wl**0.5 * np.exp(-2j * np.pi / wl * np.sqrt(x0_arr**2 + (focus + defocus)**2)) / (x0_arr**2 + (focus + defocus)**2)**0.75)/n_x

h_f = np.fft.fftshift(h_f)
u0_f = np.fft.fftshift(u0_f)

from scipy.signal import fftconvolve

fa =1

u1 = np.exp(1j * np.pi * n_arr**2 * fa / n_x) * fftconvolve(np.exp(1j * np.pi * n_arr**2 * fa / n_x) * h_f * u0_f,np.exp(-1j * np.pi * n_arr**2 * fa / n_x),mode = 'same')

%matplotlib widget
#plt.plot(np.abs(np.exp(-1j * np.pi * n_arr**2 * fa / n_x)))
#plt.plot(np.abs(u1))
#plt.plot(np.unwrap(np.imag(u1)))
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
#axes[0].plot(np.abs(u1_x[500:2200]))
axes[0].plot(np.abs(u1))
axes[0].set_title('Lens_x Amplitude', fontsize=14)

#axes[1].plot(np.unwrap(np.imag(u1_x[500:2200])))
axes[1].plot(np.unwrap(np.angle(u1)))
axes[1].set_title('Lens_x Phase', fontsize=14)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Lens_x Phase')

In [9]:
n_x = 1000000
n_arr = np.arange(-n_x // 2, n_x // 2)
print(ap_x)
x0_arr = n_arr * ap_x / n_x
u0 = np.exp(1j * np.pi * x0_arr**2 / wl / focus + 1e9j * alpha * (x0_arr / focus)**3)
u0_f = np.fft.fft(u0)
h_0 = ap_x / n_x * (focus + defocus) / 1j / wl**0.5 * \
              np.exp(-2j * np.pi / wl * np.sqrt(x0_arr**2 + (focus + defocus)**2)) / \
              (x0_arr**2 + (focus + defocus)**2)**0.75
h_f = np.fft.fft(h_0)
print(h_f.shape)
h_f = np.fft.fftshift(h_f)
u0_f = np.fft.fftshift(u0_f)
%matplotlib widget
#plt.plot(n_arr)
plt.plot(np.abs(u0_f))


4e-05
(1000000,)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [10]:

from scipy.signal import fftconvolve
fa = 1
wm = np.exp(-1j * np.pi * n_arr**2 * fa / n_x)
wm_le = np.exp(-1j * np.pi * n_arr**2 * fa / n_x)
hm_le = np.exp(1j * np.pi * n_arr**2 * fa / n_x)
u1 = wm * fftconvolve(wm_le * h_f * u0_f,
                hm_le, mode='same')
print(u1.shape)
%matplotlib widget
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
#axes[0].plot(np.abs(u1_x[500:2200]))
axes[0].plot(np.abs(u1))
axes[0].set_title('Lens_x Amplitude', fontsize=14)

#axes[1].plot(np.unwrap(np.imag(u1_x[500:2200])))
axes[1].plot(np.unwrap(np.angle(u1)))
axes[1].set_title('Lens_x Phase', fontsize=14)

(1000000,)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Text(0.5, 1.0, 'Lens_x Phase')